# PyTorch LSTM with TensorFlow-like initialization

My purpose for this notebook is to reproduce,

Dmitry Uarov: https://www.kaggle.com/dmitryuarov/ventilator-pressure-eda-lstm-0-189/notebook

with PyTorch (public LB 0.189). This sounds easy, but not always. I first got a significantly worse score ~ 0.3 using the same features and the model. Since I have heard that the weight initializations are different between PyTorch and TensorFlow, I am trying to make them as similar as I can in this notebook.

The weight initializations are as follows, according to the official documents
([Keras](https://keras.io/api/layers/recurrent_layers/lstm/), 
[PyTorch](https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html)):

| Parameter | TensorFlow/Keras | PyTorch |
| ---       | --- | --- |
| weight_ih | xavier uniform | uniform √hidden_size |
| weight_hh | orthogonal     | same as above                    |
| bias      | 1 for forget gate, 0 other wise | same as above    |
| linear    | xavier uniform | uniform √input_size |

I wrote `_reinitialize()` in class `Model`, which is the main content of this notebook.

For me, using Xavier uniform for the fully connected (linear) after the LSTM was most important (which 
looks least important to me, though). TensorFlow initialization scheme for LSTM helped, too.

Remaining uncertainties:

* One LSTM weight is actually 4 matrices packed in one tensor. Should I initialize 4 matrices separately?
* Two biases bi and bh in Pytorch LSTM seem redundant. For the forget gate, I only set one of them to 1 because I saw somewhere that Keras have only one bias, but I am not sure.

Change log
* Version 3: Public score is computed from 5 models (5 folds) trained locally. It was from 2 folds trained in notebook in version 2.

In [1]:
import numpy as np
import pandas as pd
import math
import time
import pickle
import argparse
import sklearn.preprocessing
import torch
import torch.nn as nn
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.model_selection import KFold

debug = False

def set_seed(seed=42):
    np.random.seed(seed)
    torch.manual_seed(seed)
set_seed(42)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

## Features and Dataset

From: https://www.kaggle.com/dmitryuarov/ventilator-pressure-eda-lstm-0-189/notebook

In [2]:
def create_features(df):
    df = df.copy()
    df['area'] = df['time_step'] * df['u_in']
    df['area'] = df.groupby('breath_id')['area'].cumsum()

    df['u_in_cumsum'] = (df['u_in']).groupby(df['breath_id']).cumsum()

    df['u_in_lag2'] = df['u_in'].shift(2).fillna(0)
    df['u_in_lag4'] = df['u_in'].shift(4).fillna(0)

    df['R'] = df['R'].astype(str)
    df['C'] = df['C'].astype(str)
    df = pd.get_dummies(df)

    g = df.groupby('breath_id')['u_in']
    df['ewm_u_in_mean'] = g.ewm(halflife=10).mean()\
                           .reset_index(level=0, drop=True)
    df['ewm_u_in_std'] = g.ewm(halflife=10).std()\
                          .reset_index(level=0, drop=True)
    df['ewm_u_in_corr'] = g.ewm(halflife=10).corr()\
                           .reset_index(level=0, drop=True)

    df['rolling_10_mean'] = g.rolling(window=10, min_periods=1).mean()\
                             .reset_index(level=0, drop=True)
    df['rolling_10_max'] = g.rolling(window=10, min_periods=1).max()\
                            .reset_index(level=0, drop=True)
    df['rolling_10_std'] = g.rolling(window=10, min_periods=1).std()\
                            .reset_index(level=0, drop=True)

    df['expand_mean'] = g.expanding(2).mean()\
                         .reset_index(level=0, drop=True)
    df['expand_max'] = g.expanding(2).max()\
                        .reset_index(level=0, drop=True)
    df['expand_std'] = g.expanding(2).std()\
                        .reset_index(level=0, drop=True)
    df = df.fillna(0)

    df.drop(['id', 'breath_id'], axis=1, inplace=True)
    if 'pressure' in df.columns:
        df.drop('pressure', axis=1, inplace=True)

    return df


class Dataset(torch.utils.data.Dataset):
    def __init__(self, X, y, w):
        if y is None:
            y = np.zeros(len(X), dtype=np.float32)

        self.X = X.astype(np.float32)
        self.y = y.astype(np.float32)
        self.w = w.astype(np.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, i):
        return self.X[i], self.y[i], self.w[i]

In [4]:
n = 100*1024 if debug else None

di = "/media/vincentwang/Backup/kaggle_data/ventilator-pressure-prediction/"
train = pd.read_csv(di + 'train.csv', nrows=n)
test = pd.read_csv(di + 'test.csv', nrows=n)
submit = pd.read_csv(di + 'sample_submission.csv', nrows=n)


input_size = 22#X_all.shape[2]

#print(len(X_all))

# Model

In [5]:
class Model(nn.Module):
    def __init__(self, input_size):
        hidden = [400, 300, 200, 100]
        super().__init__()
        self.lstm1 = nn.LSTM(input_size, hidden[0],
                             batch_first=True, bidirectional=True)
        self.lstm2 = nn.LSTM(2 * hidden[0], hidden[1],
                             batch_first=True, bidirectional=True)
        self.lstm3 = nn.LSTM(2 * hidden[1], hidden[2],
                             batch_first=True, bidirectional=True)
        self.lstm4 = nn.LSTM(2 * hidden[2], hidden[3],
                             batch_first=True, bidirectional=True)
        self.fc1 = nn.Linear(2 * hidden[3], 50)
        self.selu = nn.SELU()
        self.fc2 = nn.Linear(50, 1)
        self._reinitialize()

    def _reinitialize(self):
        """
        Tensorflow/Keras-like initialization
        """
        for name, p in self.named_parameters():
            if 'lstm' in name:
                if 'weight_ih' in name:
                    nn.init.xavier_uniform_(p.data)
                elif 'weight_hh' in name:
                    nn.init.orthogonal_(p.data)
                elif 'bias_ih' in name:
                    p.data.fill_(0)
                    # Set forget-gate bias to 1
                    n = p.size(0)
                    p.data[(n // 4):(n // 2)].fill_(1)
                elif 'bias_hh' in name:
                    p.data.fill_(0)
            elif 'fc' in name:
                if 'weight' in name:
                    nn.init.xavier_uniform_(p.data)
                elif 'bias' in name:
                    p.data.fill_(0)

    def forward(self, x):
        x, _ = self.lstm1(x)
        x, _ = self.lstm2(x)
        x, _ = self.lstm3(x)
        x, _ = self.lstm4(x)
        x = self.fc1(x)
        x = self.selu(x)
        x = self.fc2(x)

        return x

In [6]:
model = Model(input_size)
for name, p in model.named_parameters():
    print('%-32s %s' % (name, tuple(p.shape)))

lstm1.weight_ih_l0               (1600, 22)
lstm1.weight_hh_l0               (1600, 400)
lstm1.bias_ih_l0                 (1600,)
lstm1.bias_hh_l0                 (1600,)
lstm1.weight_ih_l0_reverse       (1600, 22)
lstm1.weight_hh_l0_reverse       (1600, 400)
lstm1.bias_ih_l0_reverse         (1600,)
lstm1.bias_hh_l0_reverse         (1600,)
lstm2.weight_ih_l0               (1200, 800)
lstm2.weight_hh_l0               (1200, 300)
lstm2.bias_ih_l0                 (1200,)
lstm2.bias_hh_l0                 (1200,)
lstm2.weight_ih_l0_reverse       (1200, 800)
lstm2.weight_hh_l0_reverse       (1200, 300)
lstm2.bias_ih_l0_reverse         (1200,)
lstm2.bias_hh_l0_reverse         (1200,)
lstm3.weight_ih_l0               (800, 600)
lstm3.weight_hh_l0               (800, 200)
lstm3.bias_ih_l0                 (800,)
lstm3.bias_hh_l0                 (800,)
lstm3.weight_ih_l0_reverse       (800, 600)
lstm3.weight_hh_l0_reverse       (800, 200)
lstm3.bias_ih_l0_reverse         (800,)
lstm3.bias_hh_l0_r

# Training

In [7]:
criterion = torch.nn.L1Loss()

def evaluate(model, loader_val):
    tb = time.time()
    was_training = model.training
    model.eval()

    loss_sum = 0
    score_sum = 0
    n_sum = 0
    y_pred_all = []

    for ibatch, (x, y, w) in enumerate(loader_val):
        n = y.size(0)
        x = x.to(device)
        y = y.to(device)
        w = w.to(device)

        with torch.no_grad():
            y_pred = model(x).squeeze()

        loss = criterion(y_pred, y)

        n_sum += n
        loss_sum += n*loss.item()
        
        y_pred_all.append(y_pred.cpu().detach().numpy())

    loss_val = loss_sum / n_sum

    model.train(was_training)

    d = {'loss': loss_val,
         'time': time.time() - tb,
         'y_pred': np.concatenate(y_pred_all, axis=0)}

    return d

In [10]:
nfold = 5
kfold = KFold(n_splits=nfold, shuffle=True, random_state=228)
epochs = 2 if debug else 300
lr = 5e-4
batch_size = 256
max_grad_norm = 1000
log = {}

for ifold, (idx_train, idx_val) in enumerate(kfold.split(train)):
    print('Fold %d' % ifold)
    tb = time.time()
    model = Model(input_size)
    model.to(device)
    model.train()

    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    scheduler = ReduceLROnPlateau(optimizer, factor=0.5, patience=10)

    train_fold, val_fold = train.iloc[idx_train,:],train.iloc[idx_val,:]
    features = create_features(train_fold)
    rs = sklearn.preprocessing.RobustScaler()
    features = rs.fit_transform(features)  # => np.ndarray

    X_train = features.reshape(-1, 80, features.shape[-1])
    y_train = train_fold.pressure.values.reshape(-1, 80)
    w_train = 1 - train_fold.u_out.values.reshape(-1, 80)  # weights for the score, but not used in this notebook
    
    X_val = create_features(val_fold)
    X_val = rs.transform(X_val)
    X_val = X_val.reshape(-1, 80, features.shape[-1])
    y_val = val_fold.pressure.values.reshape(-1, 80)
    w_val = 1 - val_fold.u_out.values.reshape(-1, 80)  # weights for the score, but not used in this notebook
    
    #     X_train = X_all[idx_train]
    #     y_train = y_all[idx_train]
    #     w_train = w_all[idx_train]

    #     X_val = X_all[idx_val]
    #     y_val = y_all[idx_val]
    #     w_val = w_all[idx_val]

    dataset_train = Dataset(X_train, y_train, w_train)
    dataset_val = Dataset(X_val, y_val, w_val)
    loader_train = torch.utils.data.DataLoader(dataset_train, shuffle=True,
                         batch_size=batch_size, drop_last=True)
    loader_val = torch.utils.data.DataLoader(dataset_val, shuffle=False,
                         batch_size=batch_size, drop_last=False)

    losses_train = []
    losses_val = []
    lrs = []
    time_val = 0
    best_score = np.inf
   
    print('epoch loss_train loss_val lr time')
    for iepoch in range(epochs):
        loss_train = 0
        n_sum = 0
        
        for ibatch, (x, y, w) in enumerate(loader_train):
            n = y.size(0)
            x = x.to(device)
            y = y.to(device)

            optimizer.zero_grad()

            y_pred = model(x).squeeze()

            loss = criterion(y_pred, y)
            loss_train += n*loss.item()
            n_sum += n

            loss.backward()
            grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)

            optimizer.step()

        val = evaluate(model, loader_val)
        loss_val = val['loss']
        time_val += val['time']

        losses_train.append(loss_train / n_sum)
        losses_val.append(val['loss'])
        lrs.append(optimizer.param_groups[0]['lr'])

        print('%3d %9.6f %9.6f %7.3e %7.1f %6.1f' %
              (iepoch + 1,
               losses_train[-1], losses_val[-1], 
               lrs[-1], time.time() - tb, time_val))

        scheduler.step(losses_val[-1])


    ofilename = 'model%d.pth' % ifold
    torch.save(model.state_dict(), ofilename)
    print(ofilename, 'written')

    log['fold%d' % ifold] = {
        'loss_train': np.array(losses_train),
        'loss_val': np.array(losses_val),
        'learning_rate': np.array(lrs),
        'y_pred': val['y_pred'],
        'idx': idx_val
    }
    
    if ifold >= 1: # due to time limit
        break


Fold 0
epoch loss_train loss_val lr time
  1  1.905934  3.746136 5.000e-04   176.6    9.2
  2  1.063023  3.650968 5.000e-04   303.4   18.6
  3  0.932291  3.527099 5.000e-04   430.6   27.8
  4  0.871136  3.281261 5.000e-04   557.8   37.1
  5  0.827874  3.058842 5.000e-04   687.4   46.9
  6  0.797389  3.062494 5.000e-04   815.3   56.3
  7  0.776537  2.898428 5.000e-04   943.4   65.6
  8  0.756011  2.784392 5.000e-04  1071.3   75.3
  9  0.730173  2.718992 5.000e-04  1199.2   84.6
 10  0.712961  2.622559 5.000e-04  1328.3   94.0
 11  0.704738  2.590038 5.000e-04  1458.3  103.6
 12  0.682354  2.553261 5.000e-04  1588.9  113.1
 13  0.669544  2.563180 5.000e-04  1718.6  122.4
 14  0.667309  2.463154 5.000e-04  1847.4  131.8
 15  0.647801  2.395836 5.000e-04  1979.0  141.5
 16  0.642287  2.407362 5.000e-04  2108.4  151.0
 17  0.627489  2.360328 5.000e-04  2236.9  160.4
 18  0.633536  2.385020 5.000e-04  2365.9  169.7
 19  0.621331  2.330669 5.000e-04  2494.4  179.1
 20  0.602168  2.298681 5.00

168  0.252533  1.959643 4.883e-07 22108.7 1610.3
169  0.252583  1.959097 4.883e-07 22235.5 1619.6
170  0.252569  1.959496 2.441e-07 22362.3 1628.9
171  0.252497  1.959581 2.441e-07 22489.1 1638.1
172  0.252486  1.959463 2.441e-07 22615.7 1647.4
173  0.252402  1.959539 2.441e-07 22742.6 1656.7
174  0.252548  1.959600 2.441e-07 22869.3 1665.9
175  0.252542  1.959644 2.441e-07 22996.1 1675.2
176  0.252369  1.959622 2.441e-07 23122.9 1684.4
177  0.252441  1.959573 2.441e-07 23249.6 1693.7
178  0.252503  1.959588 2.441e-07 23376.3 1703.0
179  0.252444  1.959537 2.441e-07 23503.2 1712.2
180  0.252485  1.959716 2.441e-07 23630.1 1721.5
181  0.252436  1.959560 1.221e-07 23757.0 1730.8
182  0.252405  1.959536 1.221e-07 23883.7 1740.0
183  0.252400  1.959683 1.221e-07 24010.5 1749.3
184  0.252435  1.959549 1.221e-07 24137.1 1758.5
185  0.252404  1.959650 1.221e-07 24263.9 1767.8
186  0.252417  1.959495 1.221e-07 24390.7 1777.0
187  0.252412  1.959487 1.221e-07 24517.3 1786.3
188  0.252437  1.959

 34  0.503807  2.142084 5.000e-04  4500.9  324.1
 35  0.507466  2.216067 5.000e-04  4628.5  333.4
 36  0.485135  2.227127 5.000e-04  4756.3  342.7
 37  0.487655  2.194291 5.000e-04  4883.9  352.1
 38  0.486788  2.072204 5.000e-04  5011.4  361.4
 39  0.486372  2.187230 5.000e-04  5139.0  370.7
 40  0.464501  2.108014 5.000e-04  5266.3  380.0
 41  0.488041  2.027762 5.000e-04  5393.7  389.3
 42  0.467884  2.090287 5.000e-04  5521.0  398.7
 43  0.454982  2.056932 5.000e-04  5648.5  408.0
 44  0.445766  2.075593 5.000e-04  5776.0  417.3
 45  0.442070  2.033876 5.000e-04  5903.4  426.6
 46  0.450679  2.010076 5.000e-04  6030.9  435.9
 47  0.450078  2.123770 5.000e-04  6158.2  445.2
 48  0.428531  2.098929 5.000e-04  6285.7  454.6
 49  0.433079  2.008862 5.000e-04  6413.3  463.9
 50  0.439071  1.942460 5.000e-04  6540.6  473.2
 51  0.443909  2.433024 5.000e-04  6668.0  482.5
 52  0.490677  1.967506 5.000e-04  6795.3  491.8
 53  0.449486  1.966113 5.000e-04  6922.8  501.2
 54  0.429941  1.942

202  0.222432  1.894826 1.221e-07 25918.0 1890.3
203  0.222433  1.894931 1.221e-07 26045.3 1899.6
204  0.222413  1.894766 1.221e-07 26172.8 1909.0
205  0.222420  1.894882 6.104e-08 26300.1 1918.3
206  0.222402  1.894812 6.104e-08 26427.6 1927.6
207  0.222369  1.894789 6.104e-08 26554.8 1936.9
208  0.222400  1.894838 6.104e-08 26682.2 1946.2
209  0.222379  1.894807 6.104e-08 26809.5 1955.5
210  0.222386  1.894877 6.104e-08 26936.7 1964.8
211  0.222414  1.894876 6.104e-08 27064.3 1974.2
212  0.222421  1.894822 6.104e-08 27191.8 1983.5
213  0.222379  1.894875 6.104e-08 27319.5 1992.8
214  0.222414  1.894774 6.104e-08 27447.3 2002.2
215  0.222405  1.894844 6.104e-08 27574.7 2011.5
216  0.222379  1.894859 3.052e-08 27702.0 2020.8
217  0.222409  1.894831 3.052e-08 27829.2 2030.1
218  0.222374  1.894854 3.052e-08 27956.4 2039.4
219  0.222378  1.894869 3.052e-08 28083.7 2048.7
220  0.222356  1.894847 3.052e-08 28210.8 2058.0
221  0.222401  1.894836 3.052e-08 28338.0 2067.3
222  0.222383  1.894

KeyboardInterrupt: 

In [ ]:
print('Fold loss_train loss_val best loss_val')
for ifold in range(2):
    d = log['fold%d' % ifold]
    print('%4d %9.6f %9.6f %9.6f' % (ifold, d['loss_train'][-1], d['loss_val'][-1], np.min(d['loss_val'])))

I trained 5 folds locally,

```
epoch loss_train loss_val
0.119303  0.184425 
0.105525  0.184154
0.109591  0.179805
0.127961  0.191654
0.141102  0.202042
```

The original TensorFlow scores at the end are,

```
loss val_loss (epoch 300)
0.1351 0.1902
0.1365 0.1897
0.1292 0.1972
0.1221 0.1970
0.1276 0.1976
```

I am satisfied with the similarity.

Note that the loss here is the overall MAE including the expiratory phase, which is not the evaluation metric.

# Predict and submit

In [9]:
features = create_features(test)
features = rs.transform(features)

X_test = features.reshape(-1, 80, features.shape[-1])
y_test = np.zeros(len(features)).reshape(-1, 80)
w_test = 1 - test.u_out.values.reshape(-1, 80)

dataset_test = Dataset(X_test, y_test, w_test)
loader_test = torch.utils.data.DataLoader(dataset_test, batch_size=batch_size)

y_pred_folds = np.zeros((len(test), 5), dtype=np.float32)
for ifold in range(5):
    model = Model(input_size)
    model.to(device)
    filename = '/kaggle/input/pytorchlstmwithtensorflowlikeinitialization/' \
               'model%d.pth' % ifold
    model.load_state_dict(torch.load(filename, map_location=device))
    model.eval()
    
    y_preds = []
    for x, y, _ in loader_test:
        x = x.to(device)
        with torch.no_grad():
            y_pred = model(x).squeeze()

        y_preds.append(y_pred.cpu().numpy())
    
    y_preds = np.concatenate(y_preds, axis=0)
    y_pred_folds[:, ifold] = y_preds.flatten()

submit.pressure = np.mean(y_pred_folds, axis=1)
submit.to_csv('submission.csv', index=False)
print('submission.csv written')

submission.csv written


Minor differences from the original, with no reason.

* Learning rate scheduler is ReduceLROnPlateau, which is used in [other notebooks](https://www.kaggle.com/tenffe/finetune-of-tensorflow-bidirectional-lstm);
* I have not implemented early stopping;
* random seeds other than kfold are not fixed in the original. Mine is not strictly deterministic either.

There are more features, loss function, or better aggrigation of nfold predictions, and so on, in public notebooks, but my goal here is to reproduce score as good as TensorFlow using same model and features.